<a href="https://colab.research.google.com/github/LarissaRodrigues99/chatAI/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar novos cursos e imersões por lá.
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre lançamentos, promoções e eventos da Alura diretamente no seu e-mail.
*   **Verificar a página da Imersão IA com Google Gemini:** Mesmo que não haja uma data definida, a página do curso pode ter informações sobre previsão de novas turmas ou um formulário para você demonstrar interesse e ser notificado quando houver novidades.

Você pode acessar a página da Imersão IA com Google Gemini neste link: [https://www.alura.com.br/imersao/ia-google-gemini](https://www.alura.com.br/imersao/ia-google-gemini)

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools": [{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nas informações encontradas, a próxima Imersão IA com Google Gemini da Alura ocorreu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram encerradas em 11 de maio de 2025.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini', 'próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: starten.tech, tecmundo.com.br, alura.com.br



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData_de_hoje: {data_de_hoje}"
  lancamento = call_agent(buscador, entrada_do_agente_buscador)
  return lancamento

In [12]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [13]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [17]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades me {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("----------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Vamos então criar o post sobre novidades me agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes sobre agentes de IA no último mês, farei algumas pesquisas no Google.
> 
> 
> Com base nas notícias mais recentes sobre agentes de IA, aqui estão alguns dos lançamentos e desenvolvimentos mais relevantes:
> 
> 1.  **Microsoft Build 2025: A Era dos Agentes de IA**: A Microsoft está focando em tornar os agentes de IA mais capazes e seguros. O Azure AI Foundry Agent Service agora está disponível, capacitando desenvolvedores a orquestrar múltiplos agentes especializados para tarefas complexas, integrando Semantic Kernel e AutoGen em um SDK único. A Microsoft visualiza um futuro onde agentes operam em contextos individuais, organizacionais e de equipe, marcando o início de uma "web agêntica aberta". (Fontes: \[1], \[3], \[6], \[7])
> 
> 2.  **IBM watsonx Orchestrate**: Durante a conferência IBM THINK, a IBM anunciou atualizações para o watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes. Há também agentes pré-construídos para áreas como RH, vendas e compras, integrados com mais de 80 aplicações empresariais. (Fonte: \[5])
> 
> 3.  **Agentes de IA Dominando o Mercado em 2025**: Há uma expectativa crescente de que os agentes de IA se tornarão dominantes em vários setores, como e-commerce, atendimento ao cliente, marketing e finanças. Empresas estão investindo fortemente nesses sistemas para automação e eficiência. (Fontes: \[2], \[4])
> 
> 4.  **Avanços e Integrações de Agentes de IA**: Empresas como IBM, Anthropic, Amazon, Google e OpenAI estão lançando atualizações que prometem revolucionar a IA, com ferramentas inovadoras e integração avançada entre agentes. Jeff Morris Jr. discute como os agentes de IA estão evoluindo para além de copilotos, assumindo funções decisórias na escolha de plataformas e negociação de preços. (Fonte: \[5])
> 
> 5.  **Novos Agentes de IA e Ferramentas de Desenvolvimento**: A Microsoft está colocando novos modelos e agentes de programação nas mãos dos desenvolvedores, introduzindo agentes com nível empresarial e adotando protocolos abertos. Ferramentas como Model Leaderboard e Model Router estão sendo introduzidas para otimizar a seleção de modelos de IA. (Fontes: \[1], \[3])
> 


----------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos e desenvolvimentos recentes sobre agentes de IA, vou criar um plano de conteúdo para um post de mídia social, destacando os pontos mais relevantes e interessantes para o público.
> 
> **Pesquisas Adicionais para Aprofundar os Temas:**
> 
> Para entender melhor o impacto e as aplicações práticas dos agentes de IA, farei algumas pesquisas adicionais no Google:
> 
> 
> Com base nas informações coletadas, o tema mais relevante para o post é:
> 
> **Tema:** A Era dos Agentes de IA: Impacto, Aplicações e Desafios
> 
> **Pontos Mais Relevantes:**
> 
> *   **Avanços Recentes:** Empresas como Microsoft, IBM, Anthropic, Amazon, Google e OpenAI estão lançando atualizações e ferramentas inovadoras para agentes de IA.
> *   **Aplicações em Diversos Setores:** E-commerce, atendimento ao cliente, marketing, finanças, RH, vendas, compras, saúde, manufatura e mais.
> *   **Benefícios:** Automação de tarefas complexas, aumento da eficiência, redução de custos, melhoria na tomada de decisões, personalização da experiência do cliente.
> *   **Desafios:** Questões éticas, privacidade e segurança de dados, viés nos modelos de IA, complexidade técnica, altos custos computacionais, necessidade de adaptação a mudanças rápidas, falta de mão de obra qualificada.
> *   **O Futuro:** Agentes de IA dominando o mercado, integrando-se a aplicações empresariais, operando em contextos individuais, organizacionais e de equipe.
> 
> **Plano para o Post:**
> 
> 1.  **Título Atraente:** "A Era dos Agentes de IA: Como Eles Estão Transformando o Mundo dos Negócios"
> 2.  **Introdução:**
>     *   Definir o que são agentes de IA e por que eles são importantes.
>     *   Mencionar os recentes lançamentos e desenvolvimentos (Microsoft Build 2025, IBM watsonx Orchestrate, etc.).
>     *   Destacar a expectativa de que os agentes de IA se tornarão dominantes em vários setores.
> 3.  **Aplicações Práticas:**
>     *   Exemplos de uso em diferentes setores (e-commerce, atendimento ao cliente, finanças, RH, saúde, etc.).
>     *   Casos de sucesso de empresas que já estão utilizando agentes de IA (eBay, Deutsche Telekom, Cosentino, Moody's, etc.).
>     *   Como os agentes de IA estão automatizando tarefas complexas e melhorando a eficiência.
> 4.  **Benefícios para as Empresas:**
>     *   Aumento da produtividade e eficiência.
>     *   Redução de custos operacionais.
>     *   Melhoria na tomada de decisões.
>     *   Personalização da experiência do cliente.
>     *   Escalabilidade e otimização de processos.
> 5.  **Desafios e Considerações Éticas:**
>     *   Questões éticas relacionadas à autonomia dos agentes de IA.
>     *   Privacidade e segurança de dados.
>     *   Viés nos modelos de IA e a necessidade de garantir a imparcialidade.
>     *   Complexidade técnica e altos custos computacionais.
>     *   Necessidade de adaptação a mudanças rápidas e de supervisão humana.
>     *   Falta de mão de obra qualificada e a importância do investimento em educação e treinamento.
> 6.  **O Futuro dos Agentes de IA:**
>     *   Como os agentes de IA vão evoluir e se integrar ainda mais em nossas vidas e nos negócios.
>     *   A "web agêntica aberta" visualizada pela Microsoft.
>     *   O impacto dos agentes de IA no mercado de trabalho e na economia.
> 7.  **Chamada para Ação:**
>     *   Incentivar os seguidores a explorar e aprender mais sobre agentes de IA.
>     *   Convidar os seguidores a compartilhar suas opiniões e experiências com agentes de IA.
>     *   Oferecer recursos adicionais, como links para artigos, webinars e cursos sobre agentes de IA.
> 8.  **Hashtags:**
>     *   \#InteligenciaArtificial \#AgentesDeIA \#AI \#Automacao \#Inovacao \#Tecnologia \#Futuro \#Negocios \#MicrosoftBuild \#IBMwatsonx
> 
> Este plano fornece uma estrutura abrangente para um post informativo e envolvente sobre a era dos agentes de IA, destacando tanto as oportunidades quanto os desafios que essa tecnologia apresenta.
> 


--------------------------------------------------------------


...

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-29' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

--------------------------------------------------------------


Exception in thread Thread-16 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
